In [ ]:
import ray
ray.init(address="auto", ignore_reinit_error=True)

In [ ]:
from AutoML import AutoML, DataNet, AutoMLDataset
import os
#DataNet().restart_datanodes()
#dataset = AutoMLDataset(name="person_classification", semantics=[113988, 113989, 113990], crops=True)#, override=True)
dataset = AutoMLDataset(name="person_classification")#, semantics=[113988, 113989, 113990], crops=True)#, override=True)

In [ ]:
saved_to_check = ['43138978_48512491',
                 '43142767_44712676',
                 '43141386_48311841',
                 '43144739_48414797',
                 '43133071_44724167',
                 '43135734_48299637',
                 '9832634_48481924',
                 '43135919_48430240',
                 '10290317_48336963',
                 '43133380_48318079',
                 '43137703_48476395',
                 '43141686_48301188']

to_check_order = []
for subset in ['test_images', 'train_images']:
    for key in dataset.dataset_header[subset].keys():
        to_check_order.append(dataset.dataset_header[subset][key][0])
        to_check_order.append(dataset.dataset_header[subset][key][-1])
    
assert to_check_order == saved_to_check

In [ ]:
# load defaulft config
import yaml

config_path = './configs/Datanet_random.yml'

with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

print(config)

In [ ]:
from test_agent_classification_service import Active_Learning_test

stages_to_eval = [int(i.split("_")[-1]) for i in os.listdir(config['PROJECT']['group_dir']) if i.startswith('Stage_')]

import time

path_stage = os.path.join(config['PROJECT']['group_dir'], 'Stage_'+str(stage))
path_checkpoitns =  os.path.join(path_stage, 'checkpoints')
path_eval = os.path.join(path_stage, 'evaluation')

list_epochs = [int(i.split(".")[1]) for i in os.listdir(path_checkpoitns) if i.startswith('checkpoint')]
max_epoch = max(list_epochs)
path_last_eval = os.path.join(path_eval, f'results_test_{max_epoch:03d}.csv')

Active_Learning = Active_Learning_test.remote(config, 
                                             dataset,
                                             4)
Active_Learning.evaluate.remote()